## This notebook will iterate over all days in the year to verify if the Azan occurs continuously at all times of the year.

In [ ]:
import collections
import datetime
import numpy as np
import os
import pandas as pd
import prayer_times
import pytz

In [ ]:
# Prayer time calculation method.
METHOD = 'Karachi'
# Times at which less than this number of cities have azan will be saved for manual verification.
MIN_CITIES_PER_TIME = 10
NUM_DAYS_PER_YEAR = 365
MINUTES_PER_DAY = 1440

# CSV with city information from GeoNames
cities_csv = ''
# PEW Country population information.
population_csv = ''

# This should point to the CSV produced from the Azan_Kepler_Vis_One_Day notebook so we can use the filtered
# list of cities to calculate times for.
filtered_dir = '/home/suleman/projects/azan/Azan/data/'
filtered_csv_name =  os.path.join(filtered_dir, 'populated_cities_karachi_filtered.csv')

In [ ]:
population_df =  pd.read_csv(population_csv)
cities_df = pd.read_csv(cities_csv, encoding='ISO-8859-1')
filtered_csv = pd.read_csv(filtered_csv_name)
cities_to_evaluate = set(filtered_csv['ASCII Name'].unique())

In [ ]:
# Split Coordinates column into lat, long
cities_df[['lat','lon']] = cities_df['Coordinates'].str.split(',',expand=True)
# Return -1 where we don't have a population value.
countries_to_population = collections.defaultdict(lambda: -1, zip(population_df['Country'], population_df['Muslims']))
# Filter cities_df to only the valid cities that we want to evaluate.
cities_df = cities_df[cities_df['ASCII Name'].isin(cities_to_evaluate)]
cities_df = cities_df.reset_index()

In [ ]:
start_date = datetime.datetime(year=2021, month=1, day=1, hour=0, minute=0, tzinfo=pytz.UTC)
prayer_names = ['fajr', 'dhuhr', 'asr', 'maghrib', 'isha']
num_errors = 0
num_correct = 0
PT = prayer_times.PrayTimes(METHOD)

invalid_rows = []
for calc_date in (start_date + datetime.timedelta(days=n) for n in range(NUM_DAYS_PER_YEAR)):
    time_to_city_ids = collections.defaultdict(list)
    calc_date_string = datetime.datetime.strftime(calc_date, "%Y-%m-%d")
    print("Calculating times for date: ", calc_date_string)
    for index, input_row in cities_df.iterrows():
        lat, lon =(float(input_row['lat']), float(input_row['lon']))
        try:
            p_times = PT.getTimes((calc_date.year,
                                   calc_date.month,
                                   calc_date.day), (lat, lon), 0)        
            for prayer_name in prayer_names:
                if p_times[prayer_name] == '-----':
                    continue
                else:
                    prayer_time = datetime.datetime.strptime(p_times[prayer_name], '%H:%M')
                    prayer_date_time = calc_date.replace(hour=prayer_time.hour, minute=prayer_time.minute)
                    time_to_city_ids[prayer_date_time].append((input_row['Geoname ID']))

            num_correct += 1                    
        except ValueError as v:
            num_errors += 1

    # Check that EVERY minute of the day has some cities with valid prayer times.
    minute_check_start_date = calc_date
    for calc_minute in (minute_check_start_date + datetime.timedelta(minutes=n) for n in range(MINUTES_PER_DAY)):
        if not(calc_minute in time_to_city_ids):
            print("Empty time found!")
            invalid_rows.append({"time": calc_minute,
                                 "min_cities": 0,
                                 "cities_in_min": []})        
        else:
            cities = time_to_city_ids[calc_minute]   
            num_cities = len(cities)
            if num_cities < MIN_CITIES_PER_TIME:    
                # print("Cities less than min specified: ", min_time, min_cities, cities_in_min)
                invalid_rows.append({"time": calc_minute,
                                     "min_cities": num_cities,
                                     "cities_in_min": cities.copy()})                


In [ ]:
# Output times with < MIN_CITIES_PER_TIME cities for manual eval.
invalid_rows_csv =  os.path.join(filtered_dir, 'invalid_rows_all_year_round.csv')
invalid_df = pd.DataFrame(invalid_rows)
invalid_df.to_csv(invalid_rows_csv, index=False)